In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import hydrant.ObjectiveFucntions.ObjectiveFucntions as obj

<frozen importlib._bootstrap>:219: RuntimeWarning: mpi4py.MPI.Status size changed, may indicate binary incompatibility. Expected 48 from C header, got 56 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: mpi4py.MPI.Datatype size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: mpi4py.MPI.Request size changed, may indicate binary incompatibility. Expected 40 from C header, got 48 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: mpi4py.MPI.Prequest size changed, may indicate binary incompatibility. Expected 40 from C header, got 48 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: mpi4py.MPI.Grequest size changed, may indicate binary incompatibility. Expected 48 from C header, got 56 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: mpi4py.MPI.Message size changed, may indicate binary incompatibility. Expected 40 from C header, got 4

In [2]:
# Example usage
# Create example data for observation
time_obs = pd.date_range(start='1990-01-01', end='2020-12-31', freq='D')
ids_obs = np.arange(1, 101)
data_obs = np.random.rand(len(time_obs), len(ids_obs))
# add np.nan values to emulate missing values in observation
data_obs[:,1] = np.nan
data_obs[:,-1] = np.nan
data_obs[1,:] = np.nan
data_obs[-1,:] = np.nan
# convert to dataframe
observation = xr.Dataset({'Discharge_obs': (['time', 'COMID'], data_obs)},
                         coords={'time': time_obs, 'COMID': ids_obs})
observation

<xarray.Dataset>
Dimensions:        (time: 11323, COMID: 100)
Coordinates:
  * time           (time) datetime64[ns] 1990-01-01 1990-01-02 ... 2020-12-31
  * COMID          (COMID) int64 1 2 3 4 5 6 7 8 9 ... 93 94 95 96 97 98 99 100
Data variables:
    Discharge_obs  (time, COMID) float64 0.6885 nan 0.5096 ... nan nan nan

In [3]:
# Create example data for simulation
time_sim = pd.date_range(start='1950-01-01 12:00:00', end='2100-12-31 12:00:00', freq='7D')
ids_sim = np.arange(50, 151)
data_sim = np.random.rand(len(time_sim), len(ids_sim))
simulation = xr.Dataset({'Discharge_sim': (['time', 'SegId'], data_sim)},
                        coords={'time': time_sim, 'SegId': ids_sim})
simulation

<xarray.Dataset>
Dimensions:        (time: 7879, SegId: 101)
Coordinates:
  * time           (time) datetime64[ns] 1950-01-01T12:00:00 ... 2100-12-26T1...
  * SegId          (SegId) int64 50 51 52 53 54 55 ... 145 146 147 148 149 150
Data variables:
    Discharge_sim  (time, SegId) float64 0.1627 0.5409 0.03447 ... 0.1775 0.1758

In [4]:
# Call the ObjectiveFunction with the example datasets
Objective = obj.ObjectiveFunction(observation,
                                  simulation,
                                  info_obs={'var': 'Discharge_obs', 'var_id': 'COMID', 'dim_id': 'COMID', 'var_time': 'time', 'dim_time': 'time'},
                                  info_sim={'var': 'Discharge_sim', 'var_id': 'SegId', 'dim_id': 'SegId', 'var_time': 'time', 'dim_time': 'time'},
                                  TimeStep='daily')

# Print the result
Objective

<xarray.Dataset>
Dimensions:  (ID: 51, time: 1617)
Coordinates:
  * ID       (ID) int64 50 51 52 53 54 55 56 57 58 ... 93 94 95 96 97 98 99 100
  * time     (time) datetime64[ns] 1990-01-07 1990-01-14 ... 2020-12-27
Data variables:
    obs      (time, ID) float64 0.5946 0.5071 0.1779 ... 0.8351 0.08859 nan
    sim      (time, ID) float64 0.4069 0.6216 0.04094 ... 0.8631 0.02708 0.5788
    KGE      (ID) float64 -0.02428 0.05533 0.009871 ... 0.01819 -0.0006395 nan
    NSE      (ID) float64 -1.038 -0.9078 -0.9406 -1.07 ... -0.9518 -1.018 nan
    RMSE     (ID) float64 0.4166 0.3951 0.4095 0.4106 ... 0.4022 0.4031 nan

In [7]:
# load data
observation = xr.open_dataset('./data/Objective/observation1.nc')
simulation = xr.open_dataset('./data/Objective/timeCOUT.nc')

# Call the ObjectiveFunction with the example datasets
Objective = obj.ObjectiveFunction(observation,
                                  simulation,
                                  info_obs={'var': 'Flow',
                                            'var_id': 'MERIT_SEG',
                                            'dim_id': 'STATION_NUMBER',
                                            'var_time': 'time',
                                            'dim_time': 'time'},
                                  info_sim={'var': 'COUT',
                                            'var_id': 'id',
                                            'dim_id': 'id',
                                            'var_time': 'time',
                                            'dim_time': 'time'},
                                  TimeStep='daily')

# Print the result
Objective

<xarray.Dataset>
Dimensions:  (ID: 3, time: 724)
Coordinates:
  * ID       (ID) int64 71022153 71027942 71028585
  * time     (time) datetime64[ns] 1981-01-06 1981-01-07 ... 1982-12-30
Data variables:
    obs      (time, ID) float64 ...
    sim      (time, ID) float32 ...
    KGE      (ID) float64 -1.986 -0.04312 0.5088
    NSE      (ID) float64 -7.004 -1.449 0.1594
    RMSE     (ID) float64 187.1 117.5 43.44